### Read the script

In [7]:
import os
from ieeg.io import get_data, raw_from_layout, save_derivative
from ieeg.navigate import crop_empty_data

In [23]:
HOME = os.path.expanduser("~")
LAB_root = os.path.join(HOME, "Box", "CoganLab")
layout = get_data("LexicalDecRepDelay", root=LAB_root)
subjects = layout.get(return_type="id", target="subject")
subject = subjects[2]

In [24]:
raw = raw_from_layout(layout.derivatives['derivatives/clean'], subject=subject, preload=False, desc='clean',extension='.edf')
# Choose preload=False to release RAM space (but )
raw = crop_empty_data(raw)

Extracting EDF parameters from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\clean\sub-D0107\ieeg\sub-D0107_task-LexicalDecRepDelay_acq-01_run-01_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\clean\sub-D0107\ieeg\sub-D0107_task-LexicalDecRepDelay_acq-01_run-01_desc-clean_events.tsv.
Reading channel info from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\clean\sub-D0107\ieeg\sub-D0107_task-LexicalDecRepDelay_acq-01_run-01_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\clean\sub-D0107\ieeg\sub-D0107_acq-01_space-ACPC_electrodes.tsv.
Not fully anonymizing info - keeping his_id, sex, and hand info
Extracting EDF parameters from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\clean\s

C:\Users\bl314\AppData\Local\miniconda3\envs\ieeg\Lib\site-packages\ieeg\io.py:113: RuntimeWarning: The unit for channel(s) Trigger has changed from NA to V.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading events from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\clean\sub-D0107\ieeg\sub-D0107_task-LexicalDecRepDelay_acq-01_run-02_desc-clean_events.tsv.
Reading channel info from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\clean\sub-D0107\ieeg\sub-D0107_task-LexicalDecRepDelay_acq-01_run-02_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\clean\sub-D0107\ieeg\sub-D0107_acq-01_space-ACPC_electrodes.tsv.
Not fully anonymizing info - keeping his_id, sex, and hand info
Extracting EDF parameters from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\clean\sub-D0107\ieeg\sub-D0107_task-LexicalDecRepDelay_acq-01_run-03_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


C:\Users\bl314\AppData\Local\miniconda3\envs\ieeg\Lib\site-packages\ieeg\io.py:113: RuntimeWarning: Omitted 592 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\bl314\AppData\Local\miniconda3\envs\ieeg\Lib\site-packages\ieeg\io.py:113: RuntimeWarning: The unit for channel(s) Trigger has changed from NA to V.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading events from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\clean\sub-D0107\ieeg\sub-D0107_task-LexicalDecRepDelay_acq-01_run-03_desc-clean_events.tsv.
Reading channel info from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\clean\sub-D0107\ieeg\sub-D0107_task-LexicalDecRepDelay_acq-01_run-03_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\clean\sub-D0107\ieeg\sub-D0107_acq-01_space-ACPC_electrodes.tsv.
Not fully anonymizing info - keeping his_id, sex, and hand info
Extracting EDF parameters from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\clean\sub-D0107\ieeg\sub-D0107_task-LexicalDecRepDelay_acq-01_run-04_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


C:\Users\bl314\AppData\Local\miniconda3\envs\ieeg\Lib\site-packages\ieeg\io.py:113: RuntimeWarning: Omitted 592 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\bl314\AppData\Local\miniconda3\envs\ieeg\Lib\site-packages\ieeg\io.py:113: RuntimeWarning: The unit for channel(s) Trigger has changed from NA to V.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading events from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\clean\sub-D0107\ieeg\sub-D0107_task-LexicalDecRepDelay_acq-01_run-04_desc-clean_events.tsv.
Reading channel info from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\clean\sub-D0107\ieeg\sub-D0107_task-LexicalDecRepDelay_acq-01_run-04_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\clean\sub-D0107\ieeg\sub-D0107_acq-01_space-ACPC_electrodes.tsv.
Not fully anonymizing info - keeping his_id, sex, and hand info


C:\Users\bl314\AppData\Local\miniconda3\envs\ieeg\Lib\site-packages\ieeg\io.py:113: RuntimeWarning: Omitted 590 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\bl314\AppData\Local\miniconda3\envs\ieeg\Lib\site-packages\ieeg\io.py:113: RuntimeWarning: The unit for channel(s) Trigger has changed from NA to V.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


### Remove bad channels
https://ieeg-pipelines.readthedocs.io/en/latest/auto_examples/plot_spectrograms.html


In [25]:
from ieeg.viz.ensemble import chan_grid
from ieeg.timefreq.utils import crop_pad
from ieeg.navigate import channel_outlier_marker, crop_empty_data
from ieeg.viz.parula import parula_map
import numpy as np
import mne

In [26]:
# plot subject
# !!!!! problems: cannot find the surface file##########
from ieeg.viz.mri import plot_subj, plot_on_average
# plot the subject brain
fig1 = plot_subj('D107B')
#C:\Users\bl314\Box\ECoG_Recon\D103
#Label every electrode
mne.viz.set_3d_view(fig1, azimuth=150, elevation=70, focalpoint="auto",
                    distance="auto")

C:\Users\bl314\AppData\Local\miniconda3\envs\ieeg\Lib\site-packages\ieeg\viz\mri.py:696: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)


In [14]:
# Remove EEG channels for D101
# eeg_channels_to_exclude = ['Fp1', 'Fp2', 'F3','F4','C3','C4','P3','P4','F7','F8','T3','T4','T5','T6','O1','O2','Fz','Cz','Pz']
# Remove EEG channels for D103 (same)
eeg_channels_to_exclude = ['F7', 'F8', 'T5', 'T6', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4',
                           'T3', 'T4', 'O1', 'O2']
# Problems: !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!Channel(s) Fz, Cz, Pz, Fp1, Fp2 not found, nothing dropped. But these channels exist in the sub-D0103_task-LexicalDecRepDelay_channels.tsv!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# Remove EEG channels for D107

#raw.drop_channels(eeg_channels_to_exclude)
#raw.load_data()

In [15]:
# mark channel outliers as bad
raw.info['bads'] = channel_outlier_marker(raw, 3, 2)
# Exclude bad channels
raw.drop_channels(raw.info['bads'])

outlier round 1 channels: ['RFOP3']
outlier round 1 channels: ['RFOP3', 'RFOP13']
outlier round 1 channels: ['RFOP3', 'RFOP13', 'RFOP14']
outlier round 1 channels: ['RFOP3', 'RFOP13', 'RFOP14', 'A1']
outlier round 1 channels: ['RFOP3', 'RFOP13', 'RFOP14', 'A1', 'A2']
outlier round 1 channels: ['RFOP3', 'RFOP13', 'RFOP14', 'A1', 'A2', 'RTMM1']
outlier round 2 channels: ['RFOP3', 'RFOP13', 'RFOP14', 'A1', 'A2', 'RTMM1', 'RFOP2']
outlier round 2 channels: ['RFOP3', 'RFOP13', 'RFOP14', 'A1', 'A2', 'RTMM1', 'RFOP2', 'RFOP11']
outlier round 2 channels: ['RFOP3', 'RFOP13', 'RFOP14', 'A1', 'A2', 'RTMM1', 'RFOP2', 'RFOP11', 'A3']
outlier round 2 channels: ['RFOP3', 'RFOP13', 'RFOP14', 'A1', 'A2', 'RTMM1', 'RFOP2', 'RFOP11', 'A3', 'RIA7']
outlier round 2 channels: ['RFOP3', 'RFOP13', 'RFOP14', 'A1', 'A2', 'RTMM1', 'RFOP2', 'RFOP11', 'A3', 'RIA7', 'RTMM2']


<RawEDF | sub-D0107_task-LexicalDecRepDelay_acq-01_run-01_desc-clean_ieeg.edf, 222 x 5048410 (2465.0 s), ~249 kB, data not loaded>

In [16]:
# CAR (common average reference) DONT DO THIS NOW
# raw.set_eeg_reference(ref_channels="average", ch_type='seeg')

In [17]:
raw.plot()

Using qt as 2D backend.


### Remove bad trials and plot spectrogram
https://ieeg-pipelines.readthedocs.io/en/latest/auto_examples/plot_spectrograms_wavelet.html

In [18]:
from ieeg.navigate import trial_ieeg, outliers_to_nan
from ieeg.calc.scaling import rescale
save_dir='C:\\Users\\bl314\\Box\\CoganLab\\IndividualMeetings\\Baishen\\ieeg_results\\lexical_delay'

In [19]:
# Epoches: https://mne.tools/stable/generated/mne.Epochs.html#mne.Epochs.__getitem__
for epoch, t in zip(
        ("Auditory_stim", "Delay", "CORRECT"),
        ((-0.5, 1.5), (-0.5, 1.5), (-0.5, 1)),
      ):
    
    # Add time point for timefrequency and crop it
    t1 = t[0] - 0.5
    t2 = t[1] + 0.5
    times = (t1, t2)
    trials = trial_ieeg(raw, epoch, times, preload=True)
    # eeg file is big. If try load multiple subjects. 
    # pointing
    
    # remove bad channels
    outliers_to_nan(trials, outliers=10)
    #Learned from Aaron's sentence rep
    #https://github.com/coganlab/SentenceRep_analysis/blob/main/analysis/check/multitaper_spec.py
    
    # Multitaper wavelet timefreq
    #freq = np.arange(10, 200., 6.)
    #spectra = spectrogram(raw, freq, 'Auditory_stim', -1.2, 1.2, 'Cue', -0.5, 0,
    #                      n_jobs=-3, verbose=10, time_bandwidth=10, n_cycles=freq/2)
    # verbose: extra messages
    # https://mne.tools/dev/generated/mne.verbose.html#mne.verbose
    #spectra = spectrogram(trials, baseline=baselines, freqs=freq, n_jobs=-3, time_bandwidth=10, n_cycles=freq/2)
    # https://mne.tools/dev/generated/mne.time_frequency.tfr_array_multitaper.html (Notes)
    #crop_pad(spectra, "0.5s")
    freq = np.linspace(0.5, 200, num=80)
    kwargs = dict(average=False, n_jobs=-3, freqs=freq, return_itc=False,
                  n_cycles=freq/2, time_bandwidth=4,
                  # n_fft=int(trials.info['sfreq'] * 2.75),
                  decim=20, )
                  # adaptive=True)
    spectra = trials.compute_tfr(method="multitaper",  **kwargs)
    crop_pad(spectra, "0.5s") # cut the first and final 0.5s, change to zero
    
    # crop the baseline
    if epoch == "Auditory_stim":
        base = spectra.copy().crop(-0.5, 0)
            # Average, and do baseline correction
        base = base.average(lambda x: np.nanmean(x, axis=0), copy=True)
        
    spectra = spectra.average(lambda x: np.nanmean(x, axis=0), copy=True)
    rescale(spectra._data, base._data, mode='ratio', axis=-1)
    
    # Save spectrograms
    #fnames = [os.path.relpath(f, layout.root) for f in good.filenames]
    #spectra.info['subject_info']['files'] = tuple(fnames)
    spectra.info['bads'] = raw.info['bads']
    #https://github.com/coganlab/SentenceRep_analysis/blob/main/analysis/check/multitaper_spec.py
    if not os.path.exists(os.path.join(save_dir, subject)):
        os.mkdir(os.path.join(save_dir, subject))
    filename = os.path.join(save_dir, subject,f'{epoch}-tfr.h5')
    mne.time_frequency.write_tfrs(filename, spectra, overwrite=True)
    #spectra.save(os.path.join(save_dir,subject, 'Auditor-avg.fif'))
    

Used Annotations descriptions: [np.str_('Auditory_stim'), np.str_('CORRECT'), np.str_('Cue'), np.str_('Delay'), np.str_('ERR_TASK_REP_YN'), np.str_('ERR_TASK_REP_YN '), np.str_('ERR_TASK_YN_REP'), np.str_('ERR_TASK_YN_REP '), np.str_('NOISY'), np.str_('Repeat'), np.str_('Speak'), np.str_('Yes/No'), np.str_('banic'), np.str_('baron'), np.str_('berin'), np.str_('bison'), np.str_('boril'), np.str_('cabin'), np.str_('caris'), np.str_('cazel'), np.str_('civic'), np.str_('civil'), np.str_('comet'), np.str_('delin'), np.str_('demon'), np.str_('devim'), np.str_('facet'), np.str_('fatal'), np.str_('focal'), np.str_('galed'), np.str_('galel'), np.str_('gapel'), np.str_('gorel'), np.str_('havel'), np.str_('haven'), np.str_('hazel'), np.str_('herib'), np.str_('janem'), np.str_('japel'), np.str_('jokel'), np.str_('kelic'), np.str_('labin'), np.str_('lamin'), np.str_('latin'), np.str_('legal'), np.str_('libel'), np.str_('likel'), np.str_('limid'), np.str_('lomic'), np.str_('madam'), np.str_('mafil')

[Parallel(n_jobs=-3)]: Using backend LokyBackend with 30 concurrent workers.
[Parallel(n_jobs=-3)]: Done  42 tasks      | elapsed:   33.5s
[Parallel(n_jobs=-3)]: Done 132 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-3)]: Done 221 out of 221 | elapsed:  1.9min finished
C:\Users\bl314\AppData\Local\Temp\ipykernel_13152\2269689154.py:40: RuntimeWarning: tmin is not in time interval. tmin is set to <class 'mne.time_frequency.tfr.EpochsTFR'>.tmin (-0.492188 s)
  base = spectra.copy().crop(-0.5, 0)


Used Annotations descriptions: [np.str_('Auditory_stim'), np.str_('CORRECT'), np.str_('Cue'), np.str_('Delay'), np.str_('ERR_TASK_REP_YN'), np.str_('ERR_TASK_REP_YN '), np.str_('ERR_TASK_YN_REP'), np.str_('ERR_TASK_YN_REP '), np.str_('NOISY'), np.str_('Repeat'), np.str_('Speak'), np.str_('Yes/No'), np.str_('banic'), np.str_('baron'), np.str_('berin'), np.str_('bison'), np.str_('boril'), np.str_('cabin'), np.str_('caris'), np.str_('cazel'), np.str_('civic'), np.str_('civil'), np.str_('comet'), np.str_('delin'), np.str_('demon'), np.str_('devim'), np.str_('facet'), np.str_('fatal'), np.str_('focal'), np.str_('galed'), np.str_('galel'), np.str_('gapel'), np.str_('gorel'), np.str_('havel'), np.str_('haven'), np.str_('hazel'), np.str_('herib'), np.str_('janem'), np.str_('japel'), np.str_('jokel'), np.str_('kelic'), np.str_('labin'), np.str_('lamin'), np.str_('latin'), np.str_('legal'), np.str_('libel'), np.str_('likel'), np.str_('limid'), np.str_('lomic'), np.str_('madam'), np.str_('mafil')

[Parallel(n_jobs=-3)]: Using backend LokyBackend with 30 concurrent workers.
[Parallel(n_jobs=-3)]: Done  42 tasks      | elapsed:   29.2s
[Parallel(n_jobs=-3)]: Done 132 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-3)]: Done 221 out of 221 | elapsed:  1.9min finished


Used Annotations descriptions: [np.str_('Auditory_stim'), np.str_('CORRECT'), np.str_('Cue'), np.str_('Delay'), np.str_('ERR_TASK_REP_YN'), np.str_('ERR_TASK_REP_YN '), np.str_('ERR_TASK_YN_REP'), np.str_('ERR_TASK_YN_REP '), np.str_('NOISY'), np.str_('Repeat'), np.str_('Speak'), np.str_('Yes/No'), np.str_('banic'), np.str_('baron'), np.str_('berin'), np.str_('bison'), np.str_('boril'), np.str_('cabin'), np.str_('caris'), np.str_('cazel'), np.str_('civic'), np.str_('civil'), np.str_('comet'), np.str_('delin'), np.str_('demon'), np.str_('devim'), np.str_('facet'), np.str_('fatal'), np.str_('focal'), np.str_('galed'), np.str_('galel'), np.str_('gapel'), np.str_('gorel'), np.str_('havel'), np.str_('haven'), np.str_('hazel'), np.str_('herib'), np.str_('janem'), np.str_('japel'), np.str_('jokel'), np.str_('kelic'), np.str_('labin'), np.str_('lamin'), np.str_('latin'), np.str_('legal'), np.str_('libel'), np.str_('likel'), np.str_('limid'), np.str_('lomic'), np.str_('madam'), np.str_('mafil')

[Parallel(n_jobs=-3)]: Using backend LokyBackend with 30 concurrent workers.
[Parallel(n_jobs=-3)]: Done  42 tasks      | elapsed:   22.4s
[Parallel(n_jobs=-3)]: Done 132 tasks      | elapsed:   56.0s
[Parallel(n_jobs=-3)]: Done 221 out of 221 | elapsed:  1.4min finished


### Multitaper spectrogram plot

#### Plot data

In [22]:
del spectra
epoch =  "CORRECT"#"Delay"# "Auditory_stim"#"CORRECT"#,#
filename = os.path.join(save_dir, subject,f'{epoch}-tfr.h5')
spectra = mne.time_frequency.read_tfrs(filename)
chan_grid(spectra, size = (20,10),vlim=(0.7, 1.4), fmax=200, show=True, cmap=parula_map,)# yscale="log")



Reading C:\Users\bl314\Box\CoganLab\IndividualMeetings\Baishen\ieeg_results\lexical_delay\D0107\CORRECT-tfr.h5 ...
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline corre

[<Figure size 2000x1000 with 60 Axes>,
 <Figure size 2000x1000 with 60 Axes>,
 <Figure size 2000x1000 with 60 Axes>,
 <Figure size 2000x1000 with 60 Axes>]